# Setup of Colab Environment
---




Every Colab runs it's own instance on cloud. We need setup workshop enviroment in those steps:  
* Setup GPU instance: Runtime ->  Change runtime type 
* Install workshop package with all requiremetns from git
* Import all packages
* Mount GDrive  

In [0]:
!pip install git+https://github.com/adamoz/colab_image_processing_workshop.git

In [0]:
!pip install tb-nightly
!pip install future

In [0]:
# Basic tools
from image_processing_workshop.utils import get_image_from_url
from collections import deque
from google.colab import drive
from google.colab import files
from shutil import rmtree
import tqdm
import pandas as pd
import numpy as np
import torch
import os

# Datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Compose, ToTensor
from torchvision.datasets import FashionMNIST

# Architecture of NN
from torch import nn
import torch.nn.functional as F
from torch.nn import Module, Sequential
from torch.nn import ReLU, Tanh, Dropout, Softmax, Linear, BatchNorm1d, Conv2d, MaxPool2d, BatchNorm2d
from torch.nn.init import xavier_uniform_, normal_

# Training
from torch.nn import MSELoss, CrossEntropyLoss, NLLLoss
from torch.optim import Adam, SGD

# Metrics
from image_processing_workshop.eval import get_results_df
from image_processing_workshop.eval import get_precision
from image_processing_workshop.eval import get_recall
from image_processing_workshop.eval import get_rec_prec
from image_processing_workshop.eval import get_accuracy
from image_processing_workshop.eval import get_false_positives

# Visualization
import matplotlib.pyplot as plt
from image_processing_workshop.visual import plot_classify, plot_image
from image_processing_workshop.visual import plot_df_examples
from image_processing_workshop.visual import plot_coocurance_matrix
from ipywidgets import interactive
import ipywidgets as ipw
from torch.utils.tensorboard import SummaryWriter

In [0]:
drive.mount('./drive', force_remount=True)

In [0]:
os.listdir('./drive/My Drive/ml_college_data')

# Work with PyTorch Datasets
---

## Custom dataset

### Creating of dataset
PyTorch provides easy mechanism to work with datasets. You just need to inherit from `torch.utils.data.Dataset` and override 2 methods:
 - `__len__` in a way that len(dataset) returns the size of the dataset.
 - `__getitem__` to support the indexing such that dataset[i] can be used to get ith sample

In [0]:
class LetterDataset(Dataset):
    """A-F Letter dataset."""
    
    def __init__(self, transform=None):
        """
        Args:
            transform (callable, optional): Optional transformation to be applied on a sample.
        """
        self.raw_data = ['A', 'B', 'C', 'D', 'E', 'F']
        self.transform = transform
        
    def __len__(self):
        return len(self.raw_data)
    
    def __getitem__(self, idx):
        sample = {'letter': self.raw_data[idx]}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [0]:
letter_dataset = LetterDataset()

In [0]:
len(letter_dataset)

In [0]:
letter_dataset[5]

### Apply transformations to dataset
We can create objects with `__call__` method applying transforamtions to data from dataset. To put more transformations together, we can use `torchvision.transforms.Compose`. PyTorch provides multiple prepared  image transformations in ``torchvision.transforms`.

In [0]:
class ToLower(object):
    def __call__(self, sample):
        return {'letter': sample['letter'].lower()}

class JoinX(object):
    def __call__(self, sample):
        return {'letter': sample['letter'] + 'X'}

In [0]:
transformations = Compose([ToLower(), JoinX()])

In [0]:
letter_dataset = LetterDataset(transform=transformations)
letter_dataset[5]

### Sampling batches from dataset
PyTorch provides iterator `torch.utils.data.DataLoader` for work with datasets based on `torch.utils.data.Dataset` class.   
It enables
 - batching the data
 - shuffling the data  
 - load the data in parallel manner using multiprocessing workers


In [0]:
data_loader = DataLoader(dataset=letter_dataset, batch_size=3, num_workers=1, shuffle=True)

In [0]:
next(iter(data_loader))

## Explore prepared dataset Fashion MNIST

In [0]:
transformations = transforms.Compose([transforms.ToTensor()])

### Training data

In [0]:
train_dataset = datasets.FashionMNIST('./drive/My Drive/ml_college_data/dataset_fashion_mnist/', download=True, train=True, transform=transformations)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

In [0]:
train_dataset.class_to_idx

In [0]:
len(train_dataset)

In [0]:
train_dataset[0]

In [0]:
next(iter(train_loader))

In [0]:
plt.subplots_adjust(wspace=1.5, hspace=2.5)
fig = plt.figure(figsize=(20,25))

img_batch, label_batch = next(iter(train_loader))
img_batch = img_batch.squeeze(dim=1).numpy()
label_batch = label_batch.numpy()
for img_id in range(100):
    ax = plt.subplot(10, 10, img_id+1)
    img = img_batch[img_id]
    
    class_id = label_batch[img_id]
    class_name = train_dataset.classes[class_id]
    ax.imshow(img , cmap='gray')
    ax.set_title(class_name)
    ax.axes.set_axis_off()

### Validation data

In [0]:
valid_dataset = datasets.FashionMNIST('./drive/My Drive/ml_college_data/dataset_fashion_mnist/', download=True, train=False, transform=transformations)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [0]:
len(valid_dataset)

In [0]:
plot_image(valid_dataset[21][0], figsize=(5, 5))

In [0]:
labels = valid_dataset.targets
class_names = list(map(lambda class_id: valid_dataset.classes[class_id], labels))
df = pd.DataFrame({'class_names': class_names, 'class_ids': labels})
df.head(10)

In [0]:
fig = plt.figure(figsize=(10, 10))
df.loc[:,'class_ids'].plot(kind='hist', width=0.5)
ax = plt.gca()
ax_ticks = ax.xaxis.set_ticks(np.arange(0.25, 9, 0.9))
ax_labels = ax.xaxis.set_ticklabels(list(valid_dataset.classes), rotation=70)

# Feed Forward Neural Network on Fashion MNIST
---

## Prepare Fashion MNIST dataset
We want to preprocess training data, specifically to have flatten shape `(28, 28) -> 784` in `torch.Tensor` format.

In [0]:
class FlattenTransform:
    def __call__(self, sample):
        return sample.reshape(-1)

In [0]:
transformations = Compose([ToTensor(), FlattenTransform()])

In [0]:
train_dataset = FashionMNIST('./drive/My Drive/ml_college_data/dataset_fashion_mnist/', download=True, train=True, 
                             transform=transformations, 
                             target_transform=None)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = FashionMNIST('./drive/My Drive/ml_college_data/dataset_fashion_mnist/', download=True, train=False, 
                             transform=transformations, 
                             target_transform=None)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [0]:
train_dataset[0]

In [0]:
next(iter(train_loader))

---

## Define feed forward neural network
In case we use `torch.nn` modules, we don't need to register tensor with `torch.nn.Parameter`.   

**Important:** Don't forget to setup `.eval()` or `.train()` modes for model to enforce proper behaviour of certain layers as `torch.nn.Dropout` or `torch.nn.BatchNorm1d`.

### Architecture

In [0]:
class FeedForwardNeuralNet(torch.nn.Module):
    def __init__(self):
        super(FeedForwardNeuralNet, self).__init__()
        
        self.layer_1 = Linear(784, 10)
        ##########################
        # TODO: Add extra layer. #
        ##########################
        
        ##########################################################
        # TODO: Prepare batch norlmalization and dropout module. #
        ##########################################################
        
    def forward(self, input_batch):
        prediction = self.layer_1(input_batch)
        ###################################################
        # TODO: Stack activation -> bn -> dropout layers. #
        ###################################################
        
        return torch.softmax(prediction, dim=1)
        ############################################################################################
        # TODO Advanced: Numeric optimization                                                      #
        #       Switch torch.softmax -> torch.log_softmax during training. Softmax leave for eval. #
        #       Use torch.nn.NLLLoss as loss (https://pytorch.org/docs/stable/nn.html#nllloss).    #
        #       Why is it cool?                                                                    #
        ############################################################################################

In [0]:
feed_forward_neural_net = FeedForwardNeuralNet()

In [0]:
feed_forward_neural_net

In [0]:
feed_forward_neural_net.state_dict()

In [0]:
images, labels = next(iter(valid_loader))

In [0]:
feed_forward_neural_net.eval()
predictions = feed_forward_neural_net(images)
feed_forward_neural_net.train()
predictions[:4]

In [0]:
torch.cuda.current_device()

In [0]:
feed_forward_neural_net = feed_forward_neural_net.cuda()
images = images.cuda()
labels = labels.cuda()

In [0]:
feed_forward_neural_net.eval()
predictions = feed_forward_neural_net(images)
feed_forward_neural_net.train()
predictions[:4]

### Optimizers and loss function

In [0]:
#######################################
# TODO(home): Try NLLLoss with log_softmax. #
#######################################
loss_fce = CrossEntropyLoss()
loss_fce

In [0]:
loss_fce(predictions, labels)

In [0]:
###############################
# TODO: Adjust learning rate. #
###############################
optimizer = SGD(feed_forward_neural_net.parameters(), lr=0.5)

###################################
# TODO: Switch optimizer to Adam. #
###################################
optimizer

### Training of neural net

In [0]:
def get_valid_acc_and_loss(model, loss_fce, valid_loader):
    accuracy = 0
    loss = 0
    was_training = model.training
    
    model.eval()
    for images, labels in valid_loader:
        images = images.cuda()
        labels = labels.cuda()
        predictions = model(images)
        accuracy += (predictions.argmax(dim=1) == labels).type(torch.FloatTensor).mean().item() 
        loss += loss_fce(predictions, labels).item()
    model.train(mode=was_training)
    return accuracy / len(valid_loader) * 100, loss / len(valid_loader)

In [0]:
get_valid_acc_and_loss(feed_forward_neural_net, loss_fce, valid_loader)

In [0]:
# Initial params setup.
epochs = 2
report_period = 100
batch_iteration = 0

# Storing of some data.
train_leak_loss = deque(maxlen=report_period)
train_loss_history = []
valid_loss_history = []
valid_acc_history = []

In [0]:
for epoch in range(epochs):
    # Setup net to train mode and go through one epoch.
    feed_forward_neural_net.train()
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()
        batch_iteration += 1
        
        ##################
        # Training Phase #
        ##################
        optimizer.zero_grad()
        predictions = feed_forward_neural_net.forward(images)
        loss = loss_fce(predictions, labels)
        loss.backward()
        optimizer.step()
        
        
        ####################
        # Validation Phase #
        ####################
        train_leak_loss.append(loss.item())
        if batch_iteration % report_period == 0:
            feed_forward_neural_net.eval()
            
            # We don't want to collect info for gradients from here.
            with torch.no_grad():
                valid_accuracy, valid_loss = get_valid_acc_and_loss(feed_forward_neural_net, loss_fce, valid_loader)
                
            print(f"Epoch: {epoch+1}/{epochs}.. ",
                  f"Train Loss: {round(np.mean(train_leak_loss), 2)}.. ",
                  f"Valid Loss: {round(valid_loss, 2)}.. ",
                  f"Valid Acc: {round(valid_accuracy, 2)}%")
            
            train_loss_history.append(np.mean(train_leak_loss))
            valid_loss_history.append(valid_loss)
            valid_acc_history.append(valid_accuracy)
                   
            feed_forward_neural_net.train()

In [0]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Cross Entropy')
plt.plot(train_loss_history, label='Train loss')
plt.plot(valid_loss_history, label='Valid loss')
plt.legend(frameon=False)

In [0]:
fig = plt.figure(figsize=(10, 10))
plt.plot(valid_acc_history, label='Valid acc')
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Acc(%)')
plt.legend(frameon=False)

## Results evaluation

In [0]:
feed_forward_neural_net.eval()
feed_forward_neural_net = feed_forward_neural_net.cpu()

### View single images and predictions

In [0]:
plot_classify(input_tensor=valid_dataset[12][0], 
              model=feed_forward_neural_net, image_shape=[28,28])

### Load reuslts to pandas df

In [0]:
df = get_results_df(feed_forward_neural_net, valid_loader)
df.head(10)

In [0]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Prediction Score')
df[df.label_class_name=='Dress'].label_class_score.hist(ax=ax)

In [0]:
plot_df_examples(df.iloc[:25], image_shape=[28, 28])

### Basic Metrics

In [0]:
get_precision(df, 'Dress')

In [0]:
get_recall(df, 'Dress')

In [0]:
get_rec_prec(df)

In [0]:
get_accuracy(df)

### False Positives

In [0]:
fp = get_false_positives(df, label_class_name='Shirt')

In [0]:
plot_df_examples(fp, image_shape=[28, 28])

In [0]:
fp = get_false_positives(df, label_class_name='Shirt', predicted_class_name='Pullover')

In [0]:
plot_df_examples(fp, image_shape=[28, 28])

### Confusion Matrix

In [0]:
plot_coocurance_matrix(df, use_log=False)

# Convolutional Neural Network on Fashion MNIST
---

## Intro to convolutional filters

### Download your favouirite image

In [0]:
url = 'https://media.wired.com/photos/5bbf72c46278de2d2123485b/master/w_582,c_limit/soyuz-1051882240.jpg'
img = get_image_from_url(url, to_grayscale=True)
img = img / 255.
plot_image(img)

### Explore prepared filters

In [0]:
initial_filter = np.array([[-1, -1, 1, 1], 
                           [-1, -1, 1, 1], 
                           [-1, -1, 1, 1], 
                           [-1, -1, 1, 1]])
filter_1 = initial_filter
filter_2 = -filter_1
filter_3 = filter_1.T
filter_4 = -filter_3
filters = np.array([filter_1, filter_2, filter_3, filter_4])

In [0]:
fig = plt.figure(figsize=(10, 5))
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))
    width, height = filters[i].shape
    
    # Add -1 1 annotations to image.
    for x in range(width):
        for y in range(height):
            ax.annotate(str(filters[i][x][y]), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if filters[i][x][y]<0 else 'black')

### Build small network initialised with those filters
In the examples, we will use `torch.nn.conv2d` https://pytorch.org/docs/stable/nn.html#conv2d

In [0]:
# In PyTorch, we have channels on 1st. So here we have 4 filters, each has 1 channel, all are shape 4x4.
filters_torch = torch.from_numpy(filters).unsqueeze(1).type(torch.DoubleTensor)
filters_torch.shape

In [0]:
img_torch = torch.from_numpy(img).unsqueeze(0).unsqueeze(1)
img_torch.shape

Convoluton filters efectively change height and width of input image that

$H_{out} = \lfloor \frac{H_{in}+2×padding[0]−dilation[0]×(kernel\_size[0]−1)−1}{stride[0]} +1 \rfloor$   
$W_{out} = \lfloor \frac{W_{in}+2×padding[1]−dilation[1]×(kernel\_size[1]−1)−1}{stride[1]} +1 \rfloor$



In [0]:
class ConvNeuralNetSimple(nn.Module):    
    def __init__(self, filters_torch):
        super(ConvNeuralNetSimple, self).__init__()
        
        height, width = filters_torch.shape[2:]
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=4, 
                                    kernel_size=(height, width), bias=False)
        self.conv_layer.weight.data = filters_torch

    def forward(self, images):
        return self.conv_layer(images)
    
conv_neural_net_simple = ConvNeuralNetSimple(filters_torch)
conv_neural_net_simple

In [0]:
img_torch.shape

In [0]:
feature_maps = conv_neural_net_simple(img_torch)
feature_maps.shape

### Visualization of conv layer feature maps

In [0]:
def vizualize_feature_maps(feature_maps, n_maps= 4):
    fig = plt.figure(figsize=(20, 20))
    
    for i in range(n_maps):
        ax = fig.add_subplot(1, n_maps, i+1, xticks=[], yticks=[])
        # grab layer outputs
        ax.imshow(np.squeeze(feature_maps[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

In [0]:
# Source img.
plt.imshow(img, cmap='gray')

# Convolution filters.
fig = plt.figure(figsize=(12, 6))
fig.subplots_adjust(left=0, right=1.5, bottom=0.8, top=1, hspace=0.05, wspace=0.05)
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))

# Feature maps.    
vizualize_feature_maps(feature_maps)

### Sensitivity of image on convolution filters

In [0]:
feature_map = feature_maps[0][0].detach().numpy()
feature_map.shape

In [0]:
plot_image(filter_1, figsize=(5,5))

In [0]:
feature_map_max = feature_map.max()
def plot_sensitivity(tolerance):
    feature_map_filtered = (feature_map >= (feature_map_max - tolerance)).astype(int)
    fig = plt.figure(figsize=(10, 10))
    im = plt.imshow(feature_map_filtered, cmap='gray')
    plt.colorbar(im, orientation='horizontal')
    plt.gca().axes.set_axis_off()
    plt.show()
    
interactive(plot_sensitivity, tolerance=ipw.FloatSlider(0.5, min=0, max=feature_map_max - 0.1, step=0.01))

## Prepare Fashion MNIST dataset

In [0]:
transformations = Compose([ToTensor()])

train_dataset = FashionMNIST('./drive/My Drive/ml_college_data/dataset_fashion_mnist/', download=True, train=True, 
                             transform=transformations, target_transform=None)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataset = FashionMNIST('./drive/My Drive/ml_college_data/dataset_fashion_mnist/', download=True, train=False, 
                             transform=transformations, target_transform=None)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [0]:
valid_dataset[0][0].shape

## Define convolutional neural network

### Architecture

In [0]:
class ConvNeuralNet(nn.Module):
    def __init__(self):
        super(ConvNeuralNet, self).__init__()
        # Variables for logging of layers shapes.
        self.shape_conv1 = None

        # 1st segment of conv with batch norm and pooling.
        self.conv1 = nn.Sequential(
            Conv2d(1, 32, (3, 3), stride=(1, 1), padding=(1, 1)),
            BatchNorm2d(32),
            ReLU(),
            MaxPool2d((2, 2), stride=(2, 2)))

        #############################################################################
        # TODO: Add another convolution blocks (64 filters) and adjsut Linear part. #
        #############################################################################
        
        # Linear output.
        self.linear = Linear(14*14*32, 10)

    def forward(self, images):
        x = self.conv1(images)
        self.shape_conv1 = x.shape
        
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = torch.softmax(x, dim=1)
        return x


conv_neural_net = ConvNeuralNet()
conv_neural_net

In [0]:
valid_dataset[0][0]

In [0]:
info = conv_neural_net.eval()

In [0]:
conv_neural_net(valid_dataset[0][0].unsqueeze(0))

In [0]:
info = conv_neural_net.train()

In [0]:
conv_neural_net.shape_conv1

### Optimizers and loss function
More on loss functions can be found here: https://pytorch.org/docs/stable/nn.html#loss-functions  
More on optimizers can be found here: https://pytorch.org/docs/stable/optim.html

In [0]:
loss_fce = CrossEntropyLoss()
loss_fce

In [0]:
optimizer = Adam(conv_neural_net.parameters())
optimizer

### Training of neural net

In [0]:
def get_valid_acc_and_loss(model, loss_fce, valid_loader):
    accuracy = 0
    loss = 0
    was_training = model.training
    
    model.eval()
    for images, labels in valid_loader:
        predictions = model(images)
        accuracy += (predictions.argmax(dim=1) == labels).type(torch.FloatTensor).mean().item() 
        loss += loss_fce(predictions, labels).item()
    model.train(mode=was_training)
    return accuracy / len(valid_loader) * 100, loss / len(valid_loader)

In [0]:
get_valid_acc_and_loss(conv_neural_net, loss_fce, valid_loader)

In [0]:
# Initial params setup.
epochs = 2
report_period = 100
batch_iteration = 0

# Storing of some data.
train_leak_loss = deque(maxlen=report_period)
train_loss_history = []
valid_loss_history = []
valid_acc_history = []

In [0]:
for epoch in range(epochs):
    # Setup net to train mode and go through one epoch.
    conv_neural_net.train()
    for images, labels in train_loader:
        batch_iteration += 1
        
        ##################
        # Training Phase #
        ##################
        optimizer.zero_grad()
        predictions = conv_neural_net.forward(images)
        loss = loss_fce(predictions, labels)
        loss.backward()
        optimizer.step()
        
        ####################
        # Validation Phase #
        ####################
        train_leak_loss.append(loss.item())
        if batch_iteration % report_period == 0:
            conv_neural_net.eval()
            
            # We don't want to collect info for gradients from here.
            with torch.no_grad():
                valid_accuracy, valid_loss = get_valid_acc_and_loss(conv_neural_net, loss_fce, valid_loader)
                
            print(f'Epoch: {epoch+1}/{epochs}.. ',
                  f"Train Loss: {round(np.mean(train_leak_loss), 2)}.. ",
                  f"Valid Loss: {round(valid_loss, 2)}.. ",
                  f"Valid Acc: {round(valid_accuracy, 2)}%")
            
            train_loss_history.append(np.mean(train_leak_loss))
            valid_loss_history.append(valid_loss)
            valid_acc_history.append(valid_accuracy)
                   
            conv_neural_net.train()

In [0]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Cross Entropy')
plt.plot(train_loss_history, label='Train loss')
plt.plot(valid_loss_history, label='Valid loss')
plt.legend(frameon=False)

In [0]:
fig = plt.figure(figsize=(10, 10))
plt.plot(valid_acc_history, label='Valid acc')
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Acc(%)')
plt.legend(frameon=False)

## Results evaluation

In [0]:
conv_neural_net.eval()


### View single images and predictions

In [0]:
plot_classify(valid_dataset[1][0], conv_neural_net)

### Load reuslts to pandas df

In [0]:
df = get_results_df(conv_neural_net, valid_loader)
df.head(10)

In [0]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Prediction Score')
df[df.label_class_name=='Dress'].label_class_score.hist(ax=ax)

In [0]:
plot_df_examples(df.iloc[:25])

### Basic Metrics

In [0]:
get_precision(df, 'Dress')

In [0]:
get_recall(df, 'Dress')

In [0]:
get_rec_prec(df)

In [0]:
get_accuracy(df)

### False Positives


In [0]:
fp = get_false_positives(df, label_class_name='Shirt')

In [0]:
plot_df_examples(fp)

In [0]:
fp = get_false_positives(df, label_class_name='Shirt', predicted_class_name='Pullover')

In [0]:
plot_df_examples(fp)

### Confusion Matrix

In [0]:
plot_coocurance_matrix(df, use_log=False)

# Tensorboard visualizations
---

Tensorboar is great tool for visualizing neural networks, its' topology and for logging values during training and evaliation. It's by default prepared for networks build with tensorflow  framework. Now it's really easy to connect tensorboard with Pytorch

In [0]:
%load_ext tensorboard

In [0]:
conv_neural_net_features = torch.nn.Sequential(*list(conv_neural_net.children())[:-1])

In [0]:
conv_neural_net_features = conv_neural_net_features.eval().cuda()

In [0]:
features = list()
labels = list()
images = list()
idx = 0
for img_batch, label_batch in tqdm.tqdm(valid_loader):
    img_features = conv_neural_net_features(img_batch.cuda())
    img_features = img_features.view(img_features.size(0), -1).cpu().detach().numpy().tolist()
    
    features += img_features
    labels += list(map(lambda l: valid_dataset.classes[l], label_batch.squeeze().detach().numpy().tolist()))
    images += img_batch.detach().numpy().tolist()
    
    idx += 1
    if idx > 15:
      break
    
features = torch.tensor(np.array(features))
images = torch.tensor(np.array(images))

In [0]:
writer = SummaryWriter()
writer.add_graph(conv_neural_net_features.cpu(), iter(valid_loader).__next__()[0])
writer.add_embedding(mat=features, metadata=labels, label_img=images)
writer.close()

In [0]:
%tensorboard --logdir ./runs